In [4]:
import pickle
import os
import torch
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
import numpy as np
import helper.training_preprocess as tp
from models.pibnn_v2 import PIBNNV2
from helper.evaluator import Evaluator
from helper.functions import visualize_dual_3d_scatter, store_truth_data, convert_std_list_to_matrix
from helper.trajectory import Trajectory
from models.stonesoup_radar_sim import StonesoupRadarSim

In [5]:
def return_sampled_df(traj_obj_in, a_sample_pct_rmv):
    traj_obj_in.apply_sampling_gaps(a_sample_pct_rmv)
    traj_obj_in.set_converted_measurement_data(measurement_conversion_data, vel=True)
    traj_obj_in.set_measurement_sigmas(radar_simulator.bearing_std, radar_simulator.elev_std,radar_simulator.range_std, radar_simulator.range_rate_std)
    sample_df = traj_obj_in.return_trajectory_df()
    return traj_obj_in.return_nn_prepared_df(sample_df)

In [6]:
# training params

incl_rr = True

X_columns = ["measured_x", "measured_y", "measured_z", "measured_vx", "measured_vy", "measured_vz",
             "measured_plus_x", "measured_plus_y", "measured_plus_z", "measured_plus_vx", "measured_plus_vy", "measured_plus_vz",
             "sigma_range", "sigma_range_rate", "sigma_elevation", "sigma_bearing", "delta_time"]

y_columns = ["truth_x", "truth_y", "truth_z", "truth_vx", "truth_vy", "truth_vz"]

noise_levels = ['low', 'mid', 'high']

In [7]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-adjust width
pd.set_option('display.max_colwidth', None)  # Show full column content

In [8]:
redo = True
for level in noise_levels:
    radar_simulator = StonesoupRadarSim(sensor_noise_level=level, include_range_rate=incl_rr)
    path_in = f"../../dataset/dataframe-readins/full_traj_data.pkl"
    save_out = f"../../dataset/dataframe-readins/cv/augmented/full_traj_data_proc_noise_{level}.pkl"
    if os.path.exists(save_out) and not redo:
        print(f"{level} filesave already exists, skipping")
        continue
    if os.path.exists(path_in):
        with open(path_in, "rb") as c:
            full_proc_traj_data = pickle.load(c)
    
    augmented_full_proc_traj_data = {}
    for traj_no, trajectory in tqdm(full_proc_traj_data.items()):
        traj_obj = Trajectory(store_truth_data(trajectory), traj_no)
        radar_meas = radar_simulator.simulate_radar_measurements(traj_obj)
        temp, measurement_conversion_data = radar_simulator.convert_radar_observations_to_cartesian(radar_meas)
        
        df1 = return_sampled_df(deepcopy(traj_obj), 0)
        df2 = return_sampled_df(deepcopy(traj_obj), 25)
        df3 = return_sampled_df(deepcopy(traj_obj), 50)
        sample_df_prep = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
        
        X_mat = sample_df_prep[X_columns].values
        y_vec = sample_df_prep[y_columns].values
        
        physics_metadata = {'inputPos': torch.tensor(sample_df_prep[["measured_x", "measured_y", "measured_z"]][:-1].to_numpy(),
                                     dtype=torch.float32),
            
                            'inputVel': torch.tensor(sample_df_prep[["measured_vx", "measured_vy", "measured_vz"]][:-1].to_numpy(),
                                     dtype=torch.float32), 
                            
                            'inputObs': torch.tensor(sample_df_prep[["measured_plus_x",
                                                                    "measured_plus_y",
                                                                    "measured_plus_z",
                                                                    "measured_plus_vx",
                                                                    "measured_plus_vy",
                                                                    "measured_plus_vz"]][:-1].to_numpy(), 
                                                                    dtype=torch.float32),
                            
                            'inputRngErr': torch.tensor(sample_df_prep[["sigma_range"]][:-1].to_numpy(), dtype=torch.float32),
            
                            'dtVec': torch.tensor(sample_df_prep[["delta_time"]][:-1].to_numpy(), dtype=torch.float32)}
        
        augmented_full_proc_traj_data[traj_no] = {}
        augmented_full_proc_traj_data[traj_no]['X_mat'] = torch.tensor(X_mat[:-1], dtype=torch.float32) 
        augmented_full_proc_traj_data[traj_no]['y_vec'] = torch.tensor(y_vec[:-1], dtype=torch.float32) 
        augmented_full_proc_traj_data[traj_no]['physics_meta_data'] = physics_metadata
        
               
    with open(save_out, "wb") as f:
        pickle.dump(augmented_full_proc_traj_data, f)
        print("Saved!")

100%|██████████| 5093/5093 [08:53<00:00,  9.54it/s]


Saved!


100%|██████████| 5093/5093 [07:05<00:00, 11.96it/s]


Saved!


100%|██████████| 5093/5093 [07:07<00:00, 11.93it/s]


Saved!
